In [204]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [205]:
#https://drive.google.com/file/d/1gIA9AC8wNx2kdepuP2Y7K5J7LnihRD1V/view?usp=sharing
downloaded = drive.CreateFile({'id':'1gIA9AC8wNx2kdepuP2Y7K5J7LnihRD1V'}) 
downloaded.GetContentFile('Ratings.csv') 
#https://drive.google.com/file/d/11duMozcad56OxTZtGrW3nY2Ir0hKU8ul/view?usp=sharing
downloaded = drive.CreateFile({'id':'11duMozcad56OxTZtGrW3nY2Ir0hKU8ul'}) 
downloaded.GetContentFile('Teleplay.csv')
#https://drive.google.com/file/d/1TFQWYDQYwBf3cEtVsU6BtUnUZrwGOreH/view?usp=sharing
downloaded = drive.CreateFile({'id':'1TFQWYDQYwBf3cEtVsU6BtUnUZrwGOreH'}) 
downloaded.GetContentFile('New_Teleplay.csv')

In [74]:
import pandas as pd
tp=pd.read_csv("Teleplay.csv")
tp=tp.drop('name',axis=1)
tp

,teleplay_id,genre,type,episodes,rating,members
0,32281,"Drama, Romance, School, Supernatural",long,1,9.37,200630
1,28977,"Action, Comedy, Historical, Parody, Samurai, S...",medium,51,9.25,114262
2,9253,"Sci-Fi, Thriller",medium,24,9.17,673572
3,9969,"Action, Comedy, Historical, Parody, Samurai, S...",medium,51,9.16,151266
4,32935,"Comedy, Drama, School, Shounen, Sports",medium,10,9.15,93351
...,...,...,...,...,...,...
10199,9316,Restricted,short,1,4.15,211
10200,5543,Restricted,short,1,4.28,183
10201,5621,Restricted,short,4,4.88,219
10202,6133,Restricted,short,1,4.98,175


In [75]:
tp['type'].unique()

array(['long', 'medium', 'short', 'Special', 'Music', 'ONA', nan],
      dtype=object)

In [76]:
rt=pd.read_csv('Ratings.csv')
rt.head()

,user_id,teleplay_id,rating
0,1,20,-1
1,1,24,-1
2,1,79,-1
3,1,226,-1
4,1,241,-1


Removing -1 values from Ratings.csv

In [77]:
rt.loc[rt['rating']==-1,'rating']= None
rt.head()

,user_id,teleplay_id,rating
0,1,20,NaN
1,1,24,NaN
2,1,79,NaN
3,1,226,NaN
4,1,241,NaN


Handling missing values

In [78]:
tp.isna().sum()

teleplay_id      0
genre           53
type            20
episodes         0
rating         193
members          0
dtype: int64

In [79]:
gen=tp['genre'].fillna("")
tp=tp.drop('genre',axis=1)
tp['genre']=gen
tp.head()

,teleplay_id,type,episodes,rating,members,genre
0,32281,long,1,9.37,200630,"Drama, Romance, School, Supernatural"
1,28977,medium,51,9.25,114262,"Action, Comedy, Historical, Parody, Samurai, S..."
2,9253,medium,24,9.17,673572,"Sci-Fi, Thriller"
3,9969,medium,51,9.16,151266,"Action, Comedy, Historical, Parody, Samurai, S..."
4,32935,medium,10,9.15,93351,"Comedy, Drama, School, Shounen, Sports"


In [80]:
print(tp['type'].value_counts())
typ=tp['type'].fillna(tp['type'].value_counts().idxmax())
tp=tp.drop('type',axis=1)
tp['type']=typ
tp.head()

medium     3131
short      2725
long       1968
Special    1397
ONA         562
Music       401
Name: type, dtype: int64


,teleplay_id,episodes,rating,members,genre,type
0,32281,1,9.37,200630,"Drama, Romance, School, Supernatural",long
1,28977,51,9.25,114262,"Action, Comedy, Historical, Parody, Samurai, S...",medium
2,9253,24,9.17,673572,"Sci-Fi, Thriller",medium
3,9969,51,9.16,151266,"Action, Comedy, Historical, Parody, Samurai, S...",medium
4,32935,10,9.15,93351,"Comedy, Drama, School, Shounen, Sports",medium


In [84]:
tp[tp['episodes']=='Unknown']=None
g=tp['episodes'].dropna()
tp=tp.drop('episodes',axis=1)
tp['episodes']=g
tp.head()

,teleplay_id,rating,members,genre,type,episodes
0,32281.0,9.37,200630.0,"Drama, Romance, School, Supernatural",long,1
1,28977.0,9.25,114262.0,"Action, Comedy, Historical, Parody, Samurai, S...",medium,51
2,9253.0,9.17,673572.0,"Sci-Fi, Thriller",medium,24
3,9969.0,9.16,151266.0,"Action, Comedy, Historical, Parody, Samurai, S...",medium,51
4,32935.0,9.15,93351.0,"Comedy, Drama, School, Shounen, Sports",medium,10


In [90]:
tp=tp.dropna()
tp.isna().sum()

teleplay_id    0
rating         0
members        0
genre          0
type           0
episodes       0
dtype: int64

Handling Outliers

In [96]:
outliers=[]
def detect_outliers(data):
    
    threshold=3
    mean = np.mean(data)
    std =np.std(data)
    
    
    for i in data:
        z_score= (i - mean)/std 
        if np.abs(z_score) > threshold:
            outliers.append(i)
    return outliers

len(outlier_pt)

215

In [103]:
outlier_pt=detect_outliers(tp['members'].astype(int))
tp.drop(tp[tp['members'].isin(outlier_pt)].index,inplace=True)

outlier_pt=detect_outliers(tp['episodes'].astype(int))
tp.drop(tp[tp['episodes'].isin(outlier_pt)].index,inplace=True)
tp

,teleplay_id,rating,members,genre,type,episodes
1,28977.0,9.25,114262.0,"Action, Comedy, Historical, Parody, Samurai, S...",medium,51
3,9969.0,9.16,151266.0,"Action, Comedy, Historical, Parody, Samurai, S...",medium,51
4,32935.0,9.15,93351.0,"Comedy, Drama, School, Shounen, Sports",medium,10
6,820.0,9.11,80679.0,"Drama, Military, Sci-Fi, Space",short,110
7,15335.0,9.10,72534.0,"Action, Comedy, Historical, Parody, Samurai, S...",long,1
...,...,...,...,...,...,...
10199,9316.0,4.15,211.0,Restricted,short,1
10200,5543.0,4.28,183.0,Restricted,short,1
10201,5621.0,4.88,219.0,Restricted,short,4
10202,6133.0,4.98,175.0,Restricted,short,1


In [ ]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
typ_label=le.fit_transform(tp['type'])
tp=tp.drop('type',axis=1)
tp['type']=typ_label
tp.head()

In [131]:
tp=tp.reset_index(drop=True)
tp.head()

,teleplay_id,rating,members,genre,episodes,type
0,28977.0,9.25,114262.0,"Action, Comedy, Historical, Parody, Samurai, S...",51,4
1,9969.0,9.16,151266.0,"Action, Comedy, Historical, Parody, Samurai, S...",51,4
2,32935.0,9.15,93351.0,"Comedy, Drama, School, Shounen, Sports",10,4
3,820.0,9.11,80679.0,"Drama, Military, Sci-Fi, Space",110,5
4,15335.0,9.10,72534.0,"Action, Comedy, Historical, Parody, Samurai, S...",1,3


In [141]:
from sklearn.preprocessing import OneHotEncoder
  
onehotencoder = OneHotEncoder()
r=pd.DataFrame(tp['type'])
#data = np.array(columnTransformer.fit_transform(r), dtype = np.str)
rated_dummies = pd.get_dummies(r.type)
type_names={}
for i in range(6):
  type_names[i]=le.inverse_transform([i])[0]
rated_dummies=rated_dummies.rename(columns=type_names)
rated_dummies.head()

,Music,ONA,Special,long,medium,short
0,0,0,0,0,1,0
1,0,0,0,0,1,0
2,0,0,0,0,1,0
3,0,0,0,0,0,1
4,0,0,0,1,0,0


In [143]:
for r in rated_dummies.columns:
  tp[r]=rated_dummies[r]
tp.head()

,teleplay_id,rating,members,genre,episodes,type,Music,ONA,Special,long,medium,short
0,28977.0,9.25,114262.0,"Action, Comedy, Historical, Parody, Samurai, S...",51,4,0,0,0,0,1,0
1,9969.0,9.16,151266.0,"Action, Comedy, Historical, Parody, Samurai, S...",51,4,0,0,0,0,1,0
2,32935.0,9.15,93351.0,"Comedy, Drama, School, Shounen, Sports",10,4,0,0,0,0,1,0
3,820.0,9.11,80679.0,"Drama, Military, Sci-Fi, Space",110,5,0,0,0,0,0,1
4,15335.0,9.10,72534.0,"Action, Comedy, Historical, Parody, Samurai, S...",1,3,0,0,0,1,0,0


In [145]:
tp=tp.drop('type',axis=1)
tp.head()

,teleplay_id,rating,members,genre,episodes,Music,ONA,Special,long,medium,short
0,28977.0,9.25,114262.0,"Action, Comedy, Historical, Parody, Samurai, S...",51,0,0,0,0,1,0
1,9969.0,9.16,151266.0,"Action, Comedy, Historical, Parody, Samurai, S...",51,0,0,0,0,1,0
2,32935.0,9.15,93351.0,"Comedy, Drama, School, Shounen, Sports",10,0,0,0,0,1,0
3,820.0,9.11,80679.0,"Drama, Military, Sci-Fi, Space",110,0,0,0,0,0,1
4,15335.0,9.10,72534.0,"Action, Comedy, Historical, Parody, Samurai, S...",1,0,0,0,1,0,0


Add average rating from the users to tp dataframe

In [149]:
rt=rt.dropna().reset_index(drop=True)
rt.head()

,user_id,teleplay_id,rating
0,1,8074,10.0
1,1,11617,10.0
2,1,11757,10.0
3,1,15451,10.0
4,2,11771,10.0


In [155]:
avg_rt=pd.DataFrame(rt.groupby('teleplay_id').mean()['rating'])
df=pd.merge(tp, avg_rt, how='inner', left_on = 'teleplay_id', right_on = 'teleplay_id')
df=df.rename(columns = {"rating_x": "rating","rating_y":"average rating"})
df.head()

,teleplay_id,rating,members,genre,episodes,Music,ONA,Special,long,medium,short,average rating
0,28977.0,9.25,114262.0,"Action, Comedy, Historical, Parody, Samurai, S...",51,0,0,0,0,1,0,9.449495
1,9969.0,9.16,151266.0,"Action, Comedy, Historical, Parody, Samurai, S...",51,0,0,0,0,1,0,9.272552
2,32935.0,9.15,93351.0,"Comedy, Drama, School, Shounen, Sports",10,0,0,0,0,1,0,9.171484
3,820.0,9.11,80679.0,"Drama, Military, Sci-Fi, Space",110,0,0,0,0,0,1,9.389788
4,15335.0,9.10,72534.0,"Action, Comedy, Historical, Parody, Samurai, S...",1,0,0,0,1,0,0,9.191430


In [164]:
from sklearn.feature_extraction.text import TfidfVectorizer
v = TfidfVectorizer()
x = v.fit_transform(df['genre'])
gen_idf=pd.DataFrame(x.toarray())
gen_idf.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46
0,0.246691,0.0,0.0,0.0,0.0,0.202397,0.0,0.0,0.000000,0.0,0.0,0.275191,0.0,0.0,0.394489,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.448498,0.0,0.0,0.0,0.0,0.0,0.55252,0.00000,0.275191,0.0,0.0,0.290979,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
1,0.246691,0.0,0.0,0.0,0.0,0.202397,0.0,0.0,0.000000,0.0,0.0,0.275191,0.0,0.0,0.394489,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.448498,0.0,0.0,0.0,0.0,0.0,0.55252,0.00000,0.275191,0.0,0.0,0.290979,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000000,0.0,0.0,0.0,0.0,0.284510,0.0,0.0,0.407470,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.00000,0.47103,0.000000,0.0,0.0,0.409031,0.0,0.000000,0.603197,0.0,0.0,0.0,0.0,0.0,0.0
3,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.365967,0.0,0.0,0.347436,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.548469,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.00000,0.00000,0.347436,0.0,0.0,0.000000,0.0,0.569058,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
4,0.246691,0.0,0.0,0.0,0.0,0.202397,0.0,0.0,0.000000,0.0,0.0,0.275191,0.0,0.0,0.394489,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.448498,0.0,0.0,0.0,0.0,0.0,0.55252,0.00000,0.275191,0.0,0.0,0.290979,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0


In [192]:
from sklearn.preprocessing import MinMaxScaler

# create a scaler object
scaler = MinMaxScaler()
# fit and transform the data
df_norm = pd.DataFrame(scaler.fit_transform(df[['members','episodes']]), columns=['members','episodes'])

for r in rated_dummies.columns:
  df_norm[r]=rated_dummies[r]


In [193]:
df_norm=df_norm.drop('Music',axis=1)


In [253]:
for i in range(47):
  df_norm[i]=gen_idf[i]
df_norm.head()

,members,episodes,ONA,Special,long,medium,short,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46
0,0.625296,0.027518,0,0,0,1,0,0.246691,0.0,0.0,0.0,0.0,0.202397,0.0,0.0,0.000000,0.0,0.0,0.275191,0.0,0.0,0.394489,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.448498,0.0,0.0,0.0,0.0,0.0,0.55252,0.00000,0.275191,0.0,0.0,0.290979,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
1,0.827857,0.027518,0,0,0,1,0,0.246691,0.0,0.0,0.0,0.0,0.202397,0.0,0.0,0.000000,0.0,0.0,0.275191,0.0,0.0,0.394489,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.448498,0.0,0.0,0.0,0.0,0.0,0.55252,0.00000,0.275191,0.0,0.0,0.290979,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
2,0.510828,0.004953,0,0,0,1,0,0.000000,0.0,0.0,0.0,0.0,0.284510,0.0,0.0,0.407470,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.00000,0.47103,0.000000,0.0,0.0,0.409031,0.0,0.000000,0.603197,0.0,0.0,0.0,0.0,0.0,0.0
3,0.441460,0.059989,0,0,0,0,1,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.365967,0.0,0.0,0.347436,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.548469,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.00000,0.00000,0.347436,0.0,0.0,0.000000,0.0,0.569058,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
4,0.396874,0.000000,0,0,1,0,0,0.246691,0.0,0.0,0.0,0.0,0.202397,0.0,0.0,0.000000,0.0,0.0,0.275191,0.0,0.0,0.394489,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.448498,0.0,0.0,0.0,0.0,0.0,0.55252,0.00000,0.275191,0.0,0.0,0.290979,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0


In [254]:
y=df['rating']
X=df_norm
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2)

In [255]:
from sklearn.linear_model import LinearRegression
regressor=LinearRegression()
regressor.fit(X_train,y_train)

y_pred=regressor.predict(X_test)

In [256]:
from sklearn.metrics import mean_squared_error

rms = mean_squared_error(y_test, y_pred, squared=False)
rms

0.7005117280273165

In [257]:
regressor=LinearRegression()
regressor.fit(X,y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [217]:
test_df=pd.read_csv("New_Teleplay.csv")
test_df

,teleplay_id,name,genre,type,episodes,rating,members
0,9345,Gakkou no Kowai Uwasa Shin: Hanako-san ga Kita!!,"Comedy, Horror, School, Supernatural",medium,33,NaN,341
1,1793,Kaze no Na wa Amnesia,"Action, Dementia, Drama, Sci-Fi",long,1,NaN,8230
2,33909,Marine Dreamin&#039;,Music,Music,1,NaN,126
3,10585,Chuunen Punk,Music,Music,1,NaN,308
4,28993,Hand Soap,"Dementia, Psychological",long,1,NaN,246
...,...,...,...,...,...,...,...
2085,3531,Shinjin Tour Conductor,Restricted,short,1,NaN,862
2086,795,Oniisama e...,"Drama, School, Shoujo, Shoujo Ai",medium,39,NaN,11424
2087,29995,The Embryo Develops into a Fetus,Dementia,ONA,1,NaN,727
2088,9973,Xi Yang Yang Yu Hui Tai Lang: Zhi Hu Hu Sheng Wei,"Adventure, Comedy, Kids",long,1,NaN,143


In [218]:
from sklearn.feature_extraction.text import TfidfVectorizer
v = TfidfVectorizer()
gen_test=test_df['genre'].fillna("")
test_df=test_df.drop('genre',axis=1)
test_df['genre']=gen_test
x = v.fit_transform(test_df['genre'])
gen_idf_test=pd.DataFrame(x.toarray())
gen_idf_test.head()


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46
0,0.000000,0.0,0.0,0.0,0.0,0.286191,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.662297,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.478387,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.500603,0.0,0.0,0.0,0.0
1,0.336085,0.0,0.0,0.0,0.0,0.000000,0.674637,0.0,0.380455,0.0,0.0,0.378923,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.000000,0.378923,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
2,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
3,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
4,0.000000,0.0,0.0,0.0,0.0,0.000000,0.678522,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.73458,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0


In [219]:
print(test_df['type'].value_counts())
typ=test_df['type'].fillna(test_df['type'].value_counts().idxmax())
test_df=test_df.drop('type',axis=1)
test_df['type']=typ
test_df.head()


medium     656
short      586
long       380
Special    279
ONA         97
Music       87
Name: type, dtype: int64


,teleplay_id,name,episodes,rating,members,genre,type
0,9345,Gakkou no Kowai Uwasa Shin: Hanako-san ga Kita!!,33,NaN,341,"Comedy, Horror, School, Supernatural",medium
1,1793,Kaze no Na wa Amnesia,1,NaN,8230,"Action, Dementia, Drama, Sci-Fi",long
2,33909,Marine Dreamin&#039;,1,NaN,126,Music,Music
3,10585,Chuunen Punk,1,NaN,308,Music,Music
4,28993,Hand Soap,1,NaN,246,"Dementia, Psychological",long


In [220]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
typ_label=le.fit_transform(test_df['type'])
test_df=test_df.drop('type',axis=1)
test_df['type']=typ_label
test_df.head()

# One Hot Encoding

from sklearn.preprocessing import OneHotEncoder
onehotencoder = OneHotEncoder()
r=pd.DataFrame(test_df['type'])
#data = np.array(columnTransformer.fit_transform(r), dtype = np.str)
rated_dummies = pd.get_dummies(r.type)
type_names={}
for i in range(6):
  type_names[i]=le.inverse_transform([i])[0]
rated_dummies=rated_dummies.rename(columns=type_names)
rated_dummies.head()

,Music,ONA,Special,long,medium,short
0,0,0,0,0,1,0
1,0,0,0,1,0,0
2,1,0,0,0,0,0
3,1,0,0,0,0,0
4,0,0,0,1,0,0


In [226]:
test_df.loc[test_df['episodes']=='Unknown','episodes']= None
test_df=test_df.drop('rating',axis=1)

In [239]:
test_df=test_df.fillna(test_df.mean())
from sklearn.preprocessing import MinMaxScaler

# create a scaler object
scaler = MinMaxScaler()
# fit and transform the data
df_norm_test = pd.DataFrame(scaler.fit_transform(test_df[['members','episodes']]), columns=['members','episodes'])

for r in rated_dummies.columns:
  df_norm_test[r]=rated_dummies[r]
df_norm_test

,members,episodes,Music,ONA,Special,long,medium,short
0,0.000415,0.024521,0,0,0,0,1,0
1,0.010355,0.000000,0,0,0,1,0,0
2,0.000144,0.000000,1,0,0,0,0,0
3,0.000373,0.000000,1,0,0,0,0,0
4,0.000295,0.000000,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...
2085,0.001071,0.000000,0,0,0,0,0,1
2086,0.014379,0.029119,0,0,0,0,1,0
2087,0.000901,0.000000,0,1,0,0,0,0
2088,0.000165,0.000000,0,0,0,1,0,0


In [250]:
df_norm_test=df_norm_test.drop('Music',axis=1)


In [258]:
for i in range(47):
  df_norm_test[i]=gen_idf_test[i]

In [260]:
df_norm_test=df_norm_test.fillna(df_norm_test.mean())
y_ans=regressor.predict(df_norm_test)

In [269]:
ans_df=pd.read_csv('New_Teleplay.csv')
ans_df.drop('rating',axis=1)
ans_df['rating']=y_ans
ans_df.to_csv("Final_rating.csv")

Neural Network

In [273]:

from keras.callbacks import ModelCheckpoint
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error 
from matplotlib import pyplot as plt
import seaborn as sb
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import warnings 

In [302]:
NN_model = Sequential()

# The Input Layer :
NN_model.add(Dense(256, kernel_initializer='normal',input_dim = X.shape[1], activation='relu'))

# The Hidden Layers :
NN_model.add(Dense(512, kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(512, kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(512, kernel_initializer='normal',activation='relu'))

# The Output Layer :
NN_model.add(Dense(1, kernel_initializer='normal',activation='linear'))

# Compile the network :
NN_model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_absolute_error'])
NN_model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_30 (Dense)             (None, 256)               14080     
_________________________________________________________________
dense_31 (Dense)             (None, 512)               131584    
_________________________________________________________________
dense_32 (Dense)             (None, 512)               262656    
_________________________________________________________________
dense_33 (Dense)             (None, 512)               262656    
_________________________________________________________________
dense_34 (Dense)             (None, 1)                 513       
Total params: 671,489
Trainable params: 671,489
Non-trainable params: 0
_________________________________________________________________


In [304]:

checkpoint_name = 'Weights-{epoch:03d}--{val_loss:.5f}.hdf5' 
checkpoint = ModelCheckpoint(checkpoint_name, monitor='val_loss', verbose = 1, save_best_only = True, mode ='auto')
callbacks_list = [checkpoint]
NN_model.fit(X, y, epochs=300, batch_size=32, validation_split = 0.2, callbacks=callbacks_list)


Epoch 1/300
201/201 [==============================] - 3s 13ms/step - loss: 1.6087 - mean_absolute_error: 1.6087 - val_loss: 0.8585 - val_mean_absolute_error: 0.8585

Epoch 00001: val_loss improved from inf to 0.85851, saving model to Weights-001--0.85851.hdf5
Epoch 2/300
201/201 [==============================] - 2s 12ms/step - loss: 0.5723 - mean_absolute_error: 0.5723 - val_loss: 0.8462 - val_mean_absolute_error: 0.8462

Epoch 00002: val_loss improved from 0.85851 to 0.84620, saving model to Weights-002--0.84620.hdf5
Epoch 3/300
201/201 [==============================] - 2s 12ms/step - loss: 0.5372 - mean_absolute_error: 0.5372 - val_loss: 0.8072 - val_mean_absolute_error: 0.8072

Epoch 00003: val_loss improved from 0.84620 to 0.80719, saving model to Weights-003--0.80719.hdf5
Epoch 4/300
201/201 [==============================] - 2s 12ms/step - loss: 0.5112 - mean_absolute_error: 0.5112 - val_loss: 0.7831 - val_mean_absolute_error: 0.7831

Epoch 00004: val_loss improved from 0.8071

KeyboardInterrupt: ignored

In [305]:
wights_file = '/content/Weights-198--0.62641.hdf5' # choose the best checkpoint 
NN_model.load_weights(wights_file) # load it
NN_model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_absolute_error'])

In [306]:

def make_submission(prediction, sub_name):
  my_submission = pd.DataFrame({'Id':pd.read_csv('test.csv').Id,'SalePrice':prediction})
  my_submission.to_csv('{}.csv'.format(sub_name),index=False)
  print('A submission file has been made')

predictions = NN_model.predict(df_norm_test)
#make_submission(predictions[:,0],'submission(NN).csv')

In [307]:
predictions

array([[6.9834657],
       [6.256876 ],
       [5.269803 ],
       ...,
       [5.078948 ],
       [5.786393 ],
       [6.5623918]], dtype=float32)

In [308]:
ans_df=pd.read_csv('New_Teleplay.csv')
ans_df.drop('rating',axis=1)
ans_df['rating']=predictions
ans_df.to_csv("Final_rating_NN.csv")